In [26]:
import tensorflow as tf
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import cv2
import pdb

In [ ]:
label={"bald_uakari":0,"black_headed_night_monkey":1,"common_squirrel_monkey":2,"japanese_macaque":3,"mantled_howler":4,
       "nilgiri_langur":5,"patas_monkey":6,"pygmy_marmoset":7,"silvery_marmoset":8, "white_headed_capuchin":9}

train_img = []
test_img = []
val_img = []
t = []
y = []
k=list(label.keys())

In [ ]:
k[label["bald_uakari"]]

In [ ]:
os.listdir("./sample")

In [ ]:
for i in range(10):
    t={0:glob('sample/bald_uakari/*.jpg'),
       1:glob('sample/black_headed_night_monkey/*.jpg'),
       2:glob('sample/common_squirrel_monkey/*.jpg'),
       3:glob('sample/japanese_macaque/*.jpg'),
       4:glob('sample/mantled_howler/*.jpg'),
       5:glob('sample/nilgiri_langur/*.jpg'),
       6:glob('sample/patas_monkey/*.jpg'),
       7:glob('sample/pygmy_marmoset/*.jpg'),
       8:glob('sample/silvery_marmoset/*.jpg'),
       9:glob('sample/white_headed_capuchin/*.jpg')}.get(i)
    
    random.shuffle(t)
    
    lent = int(len(t)*0.8)
        
    for j in range(lent):
        train_img.append([cv2.resize(cv2.imread(t[0], cv2.IMREAD_COLOR), dsize=(128, 128), interpolation=cv2.INTER_AREA), 
                      label[t[0].split('\\')[-2].split('/')[-1]]])
        train_img.append([cv2.resize(cv2.flip(cv2.imread(t[0], 1), cv2.IMREAD_COLOR), dsize=(128, 128), interpolation=cv2.INTER_AREA), 
                      label[t[0].split('\\')[-2].split('/')[-1]]])
        t.pop(0)

    lent = int(len(t)*0.5)
    for j in range(lent):
        val_img.append([cv2.resize(cv2.imread(t[0], cv2.IMREAD_COLOR), dsize=(128, 128), interpolation=cv2.INTER_AREA), 
                     label[t[0].split('\\')[-2].split('/')[-1]]])
        t.pop(0)
        
    lent = len(t)
    for j in range(lent):
        test_img.append([cv2.resize(cv2.imread(t[0], cv2.IMREAD_COLOR), dsize=(128, 128), interpolation=cv2.INTER_AREA), 
                     label[t[0].split('\\')[-2].split('/')[-1]]])
        t.pop(0)
        
    #test_img.append(cv2.flip(test_img[-1], 1))
    
    print (i, " done")


In [ ]:
len(train_img)

In [ ]:
len(val_img)

In [ ]:
len(test_img)

In [ ]:
random.shuffle(train_img)
random.shuffle(test_img)
random.shuffle(val_img)

In [ ]:
plt.imshow(train_img[10][0])

In [ ]:
k[train_img[10][1]]

In [27]:
IMG_R = 128
IMG_C = 128
IMG_CH = 3

num_class = 10


In [28]:
with tf.Graph().as_default() as tfgp:
    X = tf.placeholder(tf.float32, [None, IMG_R, IMG_C, IMG_CH])
    Y = tf.placeholder(tf.int32, [None])

    with tf.variable_scope('CNN'):
        
        layer = tf.layers.conv2d(X, 48, 5, (2, 2), padding='same', activation=tf.nn.relu)
        layer = tf.layers.max_pooling2d(layer, (2, 2), (2, 2))
        layer = tf.layers.conv2d(layer, 128, 3, (1, 1), padding='same', activation=tf.nn.relu)        
        layer = tf.layers.max_pooling2d(layer, (2, 2), (2, 2))
        layer = tf.layers.conv2d(layer, 128, 3, (1, 1), padding='same', activation=tf.nn.relu)
        layer = tf.layers.conv2d(layer, 128, 3, (1, 1), padding='same', activation=tf.nn.relu)
        layer = tf.layers.max_pooling2d(layer, (3, 3), (2, 2))
        layer = tf.layers.flatten(layer)
      
        #layer = tf.layers.dense(layer, 1024)
        out = tf.layers.dense(layer, num_class)

    with tf.variable_scope('Loss'):
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=out))

    train = tf.train.AdamOptimizer(1e-3).minimize(loss)
    saver = tf.train.Saver()

In [24]:
IMG_R = 128
IMG_C = 128
IMG_CH = 3

num_class = 10

with tf.Graph().as_default() as tfgp:
    X = tf.placeholder(tf.float32, [None, IMG_R, IMG_C, IMG_CH])
    Y = tf.placeholder(tf.int32, [None])

    with tf.variable_scope('CNN'):
        
        layer = tf.layers.conv2d(X, 48, 5, (2, 2), padding='same', activation=tf.nn.relu)
        layer = tf.layers.max_pooling2d(layer, (2, 2), (2, 2))
        layer = tf.layers.conv2d(layer, 128, 3, (1, 1), padding='same', activation=tf.nn.relu)        
        layer = tf.layers.max_pooling2d(layer, (2, 2), (2, 2))
        layer = tf.layers.conv2d(layer, 128, 3, (1, 1), padding='same', activation=tf.nn.relu)
        layer = tf.layers.conv2d(layer, 128, 3, (1, 1), padding='same', activation=tf.nn.relu)
        layer = tf.layers.max_pooling2d(layer, (3, 3), (2, 2))
        layer = tf.layers.flatten(layer)
        out = tf.layers.dense(layer, num_class)

    with tf.variable_scope('Loss'):
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=out))

    train = tf.train.AdamOptimizer(1e-3).minimize(loss)
    saver = tf.train.Saver()

In [29]:
np.sum([np.product(var.shape) for var in tfgp.get_collection('trainable_variables')]).value

416970

In [31]:
tfgp.get_collection('trainable_variables')

[<tf.Variable 'CNN/conv2d/kernel:0' shape=(5, 5, 3, 48) dtype=float32_ref>,
 <tf.Variable 'CNN/conv2d/bias:0' shape=(48,) dtype=float32_ref>,
 <tf.Variable 'CNN/conv2d_1/kernel:0' shape=(3, 3, 48, 128) dtype=float32_ref>,
 <tf.Variable 'CNN/conv2d_1/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'CNN/conv2d_2/kernel:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 <tf.Variable 'CNN/conv2d_2/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'CNN/conv2d_3/kernel:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 <tf.Variable 'CNN/conv2d_3/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'CNN/dense/kernel:0' shape=(6272, 10) dtype=float32_ref>,
 <tf.Variable 'CNN/dense/bias:0' shape=(10,) dtype=float32_ref>]

In [30]:
train_image = []
train_label = []
test_image = []
test_label = []
val_image = []
val_label = []
val = []
tra = []
cnt=0

bs=128

In [ ]:
lentrain = (len(train_img))
len(test_image)

In [ ]:
lenval = len(val_img)
for i in range(lenval):
    val_image.append(val_img[i][0])
    val_label.append(val_img[i][1])

In [ ]:
with tf.Session(graph=tfgp) as sess:
    sess.run(tf.global_variables_initializer()) 
    checkpoint = tf.train.latest_checkpoint('logs')
    if checkpoint:
        saver.restore(sess, checkpoint)
    for i in range(2000):
        #pdb.set_trace()
        for j in range(bs):
            train_image.append(train_img[0][0])
            train_label.append(train_img[0][1])
            train_img.append(train_img.pop(0))
            
        _, loss_t = sess.run([train, loss], feed_dict={X: train_image, Y: train_label})
        #print(i, l)
        tra.append(loss_t)

        for j in range(bs):
            train_image.pop(0)
            train_label.pop(0)
        
        
        if i%50==0:
            loss_t = sess.run(loss, feed_dict={X: val_image, Y: val_label})
            val.append(loss_t)
    
            print("[", cnt*50, "]", val[-1])
            cnt = cnt+1
    
    print("val: ", np.argmin(val))
    saver.save(sess, 'logs/model.ckpt', global_step=i + 1)

In [ ]:
for i in range(int(len(tra))):
    print ("[%d]" %i, tra[i])

In [ ]:
acc = 0
cnt = 1
lentest = len(test_img)


In [ ]:
lentest

In [ ]:
with tf.Session(graph=tfgp) as sess:
    sess.run(tf.global_variables_initializer())
    checkpoint = tf.train.latest_checkpoint('logs')
    if checkpoint:
        saver.restore(sess, checkpoint)
    for i in range(lentest):
        test_image.append(test_img[0][0])
        logit = sess.run(out, feed_dict={X: test_image})
        
        if np.argmax(logit[0]) == test_img[0][1]:
            acc += 1
            cv2.imwrite("./true/%s%d.jpg"%(k[test_img[0][1]],cnt), test_img[0][0])
            cnt = cnt+1
            
        else:
            print(logit[0], k[np.argmax(logit[0])], k[test_img[0][1]])
            cv2.imwrite("./false/%s%d.jpg"%(k[test_img[0][1]],cnt), test_img[0][0])
            cnt = cnt+1
            
            #cv2.imshow(k[np.argmax(logit[0])], test_image[0])
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()

        test_img.append(test_img.pop(0))
        test_image.pop(0)

    print(acc / lentest)

In [ ]:
len(val)/2